In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate() # SparkSession de forma programativa
sc = spark.sparkContext                    # SparkContext a partir de la sesión

In [6]:
rdd = sc.textFile("pdi_sales_small.csv")
rdd.count()

120240

In [7]:
ventas = rdd.map(lambda x: [x.split(";")[-1], x.split(";")])
paisesAgrupados = ventas.groupByKey()
paisesAgrupados.collect()

[('Country', <pyspark.resultiterable.ResultIterable at 0x7f3c142de610>),
 ('Canada ', <pyspark.resultiterable.ResultIterable at 0x7f3bd054f610>),
 ('France ', <pyspark.resultiterable.ResultIterable at 0x7f3c143d43d0>),
 ('France', <pyspark.resultiterable.ResultIterable at 0x7f3c18069a30>),
 ('Germany', <pyspark.resultiterable.ResultIterable at 0x7f3c18069a90>),
 ('Mexico ', <pyspark.resultiterable.ResultIterable at 0x7f3c18093670>)]

In [9]:
rdd = sc.textFile("pdi_sales_small.csv")
# Recogemos el país y las unidades de las ventas
parPais1 = rdd.map(lambda x: (x.split(";")[-1].strip(), 1))
#parPais1.collect()

In [12]:
header = parPais1.first()
parPais1SinHeader = parPais1.filter(lambda linea: linea != header)
#parPais1SinHeader.collect()

In [14]:
paisesTotal = parPais1SinHeader.reduceByKey(lambda a,b: a+b)
paisesTotal.collect()

[('Mexico', 30060), ('France', 30060), ('Germany', 30059), ('Canada', 30060)]

In [19]:
paisesUnidades = rdd.map(lambda x: (x.split(";")[-1].strip(), x.split(";")[3]))
# Le quitamos el encabezado
header = paisesUnidades.first()
paisesUnidadesSinHeader = paisesUnidades.filter(lambda linea: linea != header)
# Pasamos las unidades a un número entero
paisesUnidadesInt = paisesUnidadesSinHeader.map(lambda x: (x[0], int(x[1])))
# Reducimos por el pais y sumamos las unidades
paisesTotalUnidades = paisesUnidadesInt.reduceByKey(lambda a,b: a+b)
paisesTotalUnidades.collect()

[('Mexico', 31095), ('France', 31739), ('Germany', 31746), ('Canada', 31148)]